In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import random

In [2]:
ndata = pd.read_csv('USvideos.csv').loc[:, ["trending_date","title","channel_title","category_id","publish_time","tags","views","likes","dislikes","comment_count","thumbnail_link","comments_disabled","ratings_disabled","video_error_or_removed","description"]]

lnks = ndata.loc[:, ["title","thumbnail_link", "views"]].drop_duplicates(subset = "thumbnail_link", keep = "last").reset_index(drop = True)

In [6]:
# import urllib.request as req

# x = []
# z = []
# c = 0

# for i in lnks["thumbnail_link"]:
    
#     if c <= 5000:
        
#         try:
#             req.urlretrieve(i, "{}.jpg".format(c))
#             x.append(lnks["views"][lnks.thumbnail_link[lnks.thumbnail_link == i].index.tolist()[0]])
#             z.append(lnks["title"][lnks.thumbnail_link[lnks.thumbnail_link == i].index.tolist()[0]])
#             c += 1
            
#         except:
#             pass
    
#     else:
#         break
    
# print(c)

5001


In [19]:
# # Remove emojis    ------------[
        
# import re
        
# emoji_pattern = re.compile(
#     u"(\ud83d[\ude00-\ude4f])|"  # emoticons
#     u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
#     u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
#     u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
#     u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
#     "+", flags=re.UNICODE)
        
# #     ]------------------------


   

In [22]:
# Importando Vectorizer    ------------------------[

from sklearn.feature_extraction.text import CountVectorizer


# ]--------------------------

In [33]:
vectorizer = CountVectorizer()
vect = vectorizer.fit_transform(z)
vect = vect.toarray()

np.savetxt('z.txt', vect, delimiter=' ')    

In [44]:


# with open('z.txt', 'w') as f:
#     for item in vect:
        
# #       emoji_pattern.sub(r'', item)
#         f.write(item + "\n")

# with open('x.txt', 'w') as f:
#     for item in x:
#         f.write(str(item) + "\n")



In [ ]:
# faixa 1 = 0, 100652.364
# faixa 2 = 100652.364, 504283.8
# faixa 3 = 504283.8, 1008510.6
# faixa 4 = 1008510.6, 5127377.4
# faixa 5 = 5127377.4, 50169650.2
# faixa 6 = 50169650.2, 225211923.0

views_y = []

with open('x.txt', 'r') as f:
    for item in f:
        if int(item[:-1]) < 100000:
            views_y.append(1)
        
        if int(item[:-1]) > 100000 and int(item[:-1]) < 500000:
            views_y.append(2)
        
        if int(item[:-1]) > 500000 and int(item[:-1]) < 1000000:
            views_y.append(3)
        
        if int(item[:-1]) > 1000000 and int(item[:-1]) < 5000000:
            views_y.append(4)
        
        if int(item[:-1]) > 5000000 and int(item[:-1]) < 50000000:
            views_y.append(5)
        
        if int(item[:-1]) > 50000000 and int(item[:-1]) < 225211923.0:
            views_y.append(6)

In [ ]:
import numpy as np
import cv2

imagens_x = []

for i in range(5001):
    im = cv2.imread('{}.jpg'.format(i))
    img_crop = cv2.pyrDown(im)
    img_crop = cv2.pyrDown(img_crop)
    img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
    imagens_x.append(img_crop.flatten())

In [ ]:
shuff = list(zip(imagens_x, views_y))

random.shuffle(shuff)

imagens_x, views_y = zip(*shuff)

In [ ]:
index = round(len(imagens_x)*0.95)
imagens_x_train, imagens_x_test = imagens_x[:index], imagens_x[index:]
len(imagens_x_train)

In [ ]:
views_y
index_ = round(len(views_y)*0.95)
views_y_train, views_y_test = views_y[:index_], views_y[index_:]
len(views_y_train)

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss = "hinge", penalty = "l2", max_iter = 100)
clf.fit(imagens_x_train, views_y_train)
y_pred = clf.predict(imagens_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators = 100, max_depth = 2,random_state = 0)
clf.fit(imagens_x_train, views_y_train)
y_pred = clf.predict(imagens_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

In [ ]:
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier

rbf_feature = RBFSampler(gamma = 1, random_state = 1)
X_features = rbf_feature.fit_transform(imagens_x_train)
clf = SGDClassifier(max_iter = 100)
clf.fit(imagens_x_train, views_y_train) 
y_pred = clf.predict(imagens_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

In [ ]:
from sklearn import svm

clf = svm.SVC(gamma = 'scale')
clf.fit(imagens_x_train, views_y_train) 
y_pred = clf.predict(imagens_x_test)
print('Acurácia: {}'.format(accuracy_score(views_y_test, y_pred)))

---

In [ ]:
y_pred

### Análise Exploratória

O Objetivo do projeto é indentificar em qual faixa de views (0 - 100 mil, ...) um vídeo vai estar, dada somente sua tumbnail como entrada. Para isso, foi baixado um dataset contendo diversas variáveis como "likes", "dislikes", ..., etc. Cada coluna diz respeito a algum feature que o youtube utiliza como caracteristicas do video, como pode ser visto abaixo:

In [4]:
ndata

,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,17.14.11,The Trump Presidency: Last Week Tonight with J...,LastWeekTonight,24,2017-11-13T07:30:00.000Z,"last week tonight trump presidency|""last week ...",2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John..."
2,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,17.14.11,Nickelback Lyrics: Real or Fake?,Good Mythical Morning,24,2017-11-13T11:00:04.000Z,"rhett and link|""gmm""|""good mythical morning""|""...",343168,10172,666,2146,https://i.ytimg.com/vi/puqaWrEC7tY/default.jpg,False,False,False,Today we find out if Link is a Nickelback amat...
4,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095731,132235,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
5,17.14.11,2 Weeks with iPhone X,iJustine,28,2017-11-13T19:07:23.000Z,"ijustine|""week with iPhone X""|""iphone x""|""appl...",119180,9763,511,1434,https://i.ytimg.com/vi/gHZ1Qz0KiKM/default.jpg,False,False,False,Using the iPhone for the past two weeks -- her...
6,17.14.11,Roy Moore & Jeff Sessions Cold Open - SNL,Saturday Night Live,24,2017-11-12T05:37:17.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",2103417,15993,2445,1970,https://i.ytimg.com/vi/39idVpFF7NQ/default.jpg,False,False,False,Embattled Alabama Senate candidate Roy Moore (...
7,17.14.11,5 Ice Cream Gadgets put to the Test,CrazyRussianHacker,28,2017-11-12T21:50:37.000Z,"5 Ice Cream Gadgets|""Ice Cream""|""Cream Sandwic...",817732,23663,778,3432,https://i.ytimg.com/vi/nc99ccSXST0/default.jpg,False,False,False,Ice Cream Pint Combination Lock - http://amzn....
8,17.14.11,The Greatest Showman | Official Trailer 2 [HD]...,20th Century Fox,1,2017-11-13T14:00:23.000Z,"Trailer|""Hugh Jackman""|""Michelle Williams""|""Za...",826059,3543,119,340,https://i.ytimg.com/vi/jr9QtXwC9vc/default.jpg,False,False,False,"Inspired by the imagination of P.T. Barnum, Th..."
9,17.14.11,Why the rise of the robots won’t mean the end ...,Vox,25,2017-11-13T13:45:16.000Z,"vox.com|""vox""|""explain""|""shift change""|""future...",256426,12654,1363,2368,https://i.ytimg.com/vi/TUmyygCMMGA/default.jpg,False,False,False,"For now, at least, we have better things to wo..."


Como o nosso projeto consta somente com a imagem, cortamos o dataset em um com apenas o link das tumbnails e o número de views do vídeo. É importante dizer que os dados foram atualizados todos os dias durante o tempo, o que significa que existem videos iguais em diferentes dias, e por isso utilizamos do ferramental pandas para tirar todas as tumbnails iguais. Por conta disso, em alguns casos o número de views é diferente para o mesmo link (vídeo), optamos por considerar a ultima vez q o dataset foi atualizado, ou seja, o maior número de views de um vídeo igual.

In [5]:
lnks

,title,thumbnail_link,views
0,Dion Lewis' 103-Yd Kick Return TD vs. Denver! ...,https://i.ytimg.com/vi/9wRQljFNDW8/default.jpg,81377
1,TL;DW - Every DCEU Movie Before Justice League,https://i.ytimg.com/vi/Om_zGhJLZ5U/default.jpg,288922
2,Iraq-Iran earthquake: Deadly tremor hits borde...,https://i.ytimg.com/vi/goP4Z5wyOlM/default.jpg,34785
3,Jason Momoa Wows Hugh Grant With Some Dothraki...,https://i.ytimg.com/vi/8NHA23f7LvU/default.jpg,1496225
4,Mayo Clinic's first face transplant patient me...,https://i.ytimg.com/vi/IE-xepGLVt8/default.jpg,237307
5,Jason Momoa & Lisa Bonet: Love at First Sight,https://i.ytimg.com/vi/cOc3tsFWoRs/default.jpg,1497519
6,Mark Wahlberg's Kids Use Him for His Celeb Con...,https://i.ytimg.com/vi/wnwF1FHybDQ/default.jpg,225286
7,Camila Cabello - Havana (Vertical Video) ft. Y...,https://i.ytimg.com/vi/pz95u3UVpaM/default.jpg,5476737
8,"Owen Wilson on Meeting Julia Roberts, and Tand...",https://i.ytimg.com/vi/yuBqWcSKrCk/default.jpg,304926
9,Meryl Streep Meets Anna Wintour at Vogue,https://i.ytimg.com/vi/dW4wpGg64pE/default.jpg,1017803


Com o banco de dados pronto para ser utilizado, o próximo passo seria baixar as fotos para o computador, para otimizar o tempo na hora de trabalhar no modelo. Como muitas tumbnails não existem (vídeo removido, não existe mais foto), foi utilizado o try, e logo depois adicionado o número de views a uma lista chamada "x" (não tem relação com o modelo, o nome "x" é apenas representativo). 

    import urllib.request as req

    x = []
    c = 0

    for i in lnks["thumbnail_link"]:

        if c <= 5000:

            try:
                req.urlretrieve(i, "{}.jpg".format(c))
                x.append(lnks["views"][lnks.thumbnail_link[lnks.thumbnail_link == i].index.tolist()[0]])
                c += 1

            except:
                pass

        else:
            break

    print(c)

Além disso, como nosso modelo adota o uso de imagens para análise do numero de views, não é possível plotar histogramas ou boxplots, o que nos leva a descartar tal tipo de análise.

Foi salvo os views da lista "x" em um arquivo para nao ter de rodar novamente o código acima, o que acabaria baixando novamente as fotos.

    with open('x.txt', 'w') as f:
        for item in x:
            f.write(str(item) + "\n")

Com o y pronto (número de views) e dividido em faixas, o próximo passo seria achar algum meio de transformar as imagens baixadas em um array que representaria o x do modelo.



Para isso foi utilizado o cv2 para ler e converter as imagens em array, e prontas para serem utilizadas no modelo.

    import numpy as np
    import cv2

    imagens_x = []

    for i in range(5001):
        im = cv2.imread('{}.jpg'.format(i))
        img_crop = cv2.pyrDown(im)
        img_crop = cv2.pyrDown(img_crop)
        img_crop = cv2.cvtColor(img_crop, cv2.COLOR_BGR2RGB)
        imagens_x.append(img_crop.flatten())

Antes de testar os modelos, algumas alterações foram feitas nas listas x e y, como por exemplo o shuffle. Dividimos 95% das duas listas para treino, e o resto para classificação.
Ao entrar no site do sikit em "Flow Chart", alguns modelos foram testados seguindo a sugestão do mesmo.
Como o máximo de acurácia foi 33%, escolhemos entender melhor o que as imagens nos diziam, portanto, o proximo passo para melhorar a acurácia é criar meios de análise da imagem, para ajudar o classificador a melhorar os acertos.